Обеспеченность городскими сервисами

In [2]:
!pip install objectnat folium matplotlib mapclassify

  Using cached iduedu-0.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata (15 kB)
Using cached iduedu-0.2.2-py3-none-any.whl (25 kB)
Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl (11.1 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: iduedu
    Found existing installation: iduedu 0.1.2
    Uninstalling iduedu-0.1.2:
      Successfully uninstalled iduedu-0.1.2


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blocksnet 0.0.12 requires iduedu==0.1.2, but you have iduedu 0.2.2 which is incompatible.
blocksnet 0.0.12 requires numpy==1.23.5, but you have numpy 1.26.4 which is incompatible.
blocksnet 0.0.12 requires scikit-learn==1.2.2, but you have scikit-learn 1.6.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\meles\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install momepy


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\meles\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
!pip install objectnat==0.2.7
!pip install momepy==0.9.1



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\meles\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached momepy-0.9.1-py3-none-any.whl.metadata (1.4 kB)
Using cached momepy-0.9.1-py3-none-any.whl (1.7 MB)
  Attempting uninstall: momepy
    Found existing installation: momepy 0.7.0
    Uninstalling momepy-0.7.0:
      Successfully uninstalled momepy-0.7.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blocksnet 0.0.12 requires iduedu==0.1.2, but you have iduedu 0.2.2 which is incompatible.
blocksnet 0.0.12 requires momepy==0.7.0, but you have momepy 0.9.1 which is incompatible.
blocksnet 0.0.12 requires numpy==1.23.5, but you have numpy 1.26.4 which is incompatible.
blocksnet 0.0.12 requires scikit-learn==1.2.2, but you have scikit-learn 1.6.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\meles\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from objectnat import get_adj_matrix_gdf_to_gdf, get_intermodal_graph
import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union
from objectnat import get_service_provision

C:\Users\meles\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import momepy
import networkx as nx
import osmnx as ox

In [ ]:
# Укажи корректные пути к файлам GeoJSON
buildings = ('buildings.geojson')

# Загружаем данные
buildings = gpd.read_file(buildings)
if buildings.crs != "EPSG:3857":
      buildings = buildings.to_crs("EPSG:3857")

In [ ]:
# Укажи корректные пути к файлам GeoJSON
roads = ('roads.geojson')

# Загружаем данные
roads = gpd.read_file(roads)
if roads.crs != "EPSG:3857":
      roads = roads.to_crs("EPSG:3857")

In [ ]:
# Укажи корректные пути к файлам GeoJSON
services = ('service.geojson')

# Загружаем данные
services = gpd.read_file(services)
if services.crs != "EPSG:3857":
      services = services.to_crs("EPSG:3857")

In [307]:
buildings = buildings[buildings["is_living"] == True].copy() #Оставляем только жилые здания

In [308]:
buildings ['building_area'] = buildings['building:levels'] * buildings['footprint_area']

In [310]:
buildings['living_area'] = buildings['building_area'] * 0.7

In [311]:
buildings['population'] = buildings['living_area']/20 

In [312]:
buildings["demand"] = buildings["population"] / 1000 * 8 #спрос на сервис ()

In [313]:
polygon = unary_union([buildings.geometry.to_list() + services.geometry.to_list()]).convex_hull.buffer(
    0.001) #Получаем полигон, опоясывающий геометрии зданий и сервисов

In [314]:
# Обработка дорог
roads = roads[roads.geom_type.isin(['LineString', 'MultiLineString'])]

GAP_TOLERANCE = 1

def _get_roads(roads):
    """Обработка и объединение дорожной сети"""
    merged = roads.unary_union
    if merged.geom_type == 'MultiLineString':
        roads = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=roads.crs)
    else:
        roads = gpd.GeoDataFrame(geometry=[merged], crs=roads.crs)
    
    roads = roads.explode(index_parts=False).reset_index(drop=True)
    roads.geometry = momepy.close_gaps(roads, GAP_TOLERANCE)
    roads = roads[roads.geom_type.isin(['LineString'])]
    return roads

roads = _get_roads(roads)

In [315]:
CRS = 32636
SPEED_M_MIN = 1000

def _roads_to_graph(roads):
    """Преобразование дорог в граф"""
    graph = momepy.gdf_to_nx(roads)
    graph.graph['crs'] = roads.crs.to_epsg()
    graph = nx.DiGraph(graph)
    
    for _, _, data in graph.edges(data=True):
        geometry = data['geometry']
        data['time_min'] = geometry.length / SPEED_M_MIN
        
    for n, data in graph.nodes(data=True):
        graph.nodes[n]['x'] = n[0]  # X координата
        graph.nodes[n]['y'] = n[1]  # Y координата

    return graph

roads_G = _roads_to_graph(roads)

In [ ]:
G_intermodal = roads_G

In [317]:
#G_intermodal = get_intermodal_graph(polygon=polygon, clip_by_bounds=True) #скачиваем граф#

In [318]:
buildings.to_crs(G_intermodal.graph['crs'], inplace=True)
services.to_crs(G_intermodal.graph['crs'], inplace=True) 

In [319]:
G_intermodal = nx.convert_node_labels_to_integers(G_intermodal)

In [320]:
matrix: pd.DataFrame = get_adj_matrix_gdf_to_gdf(gdf_from=buildings,
                                                 gdf_to=services,
                                                 nx_graph=G_intermodal,
                                                 weight="time_min",
                                                 threshold=15
                                                 ) #строим матрицу доступности от сервисов до зданий

In [321]:
buildings.dropna(subset="demand", inplace=True) #На всякий случай дропаем наны из демандов, мало ли затисались

In [322]:
matrix.index = matrix.index.astype(int)
services.index = services.index.astype(int)
buildings.index = buildings.index.astype(int)
matrix.columns = matrix.columns.astype(int)
buildings["demand"] = buildings["demand"].astype(int)
services["capacity"] = services["capacity"].astype(int) #Приводим все нужные колонки, индексы и прочее к инту на всякий случай

In [323]:
build_prov, services_prov, links_prov = get_service_provision(
    buildings=buildings,
    services=services,
    adjacency_matrix=matrix,
    threshold=15,
) #Считаем обеспеченность treshold - нормативная доступность в минутах

In [324]:
build_prov.explore(column="provison_value") #смотрим что получилось

In [325]:
build_prov

,is_living,building,building:levels,build_floor_area,number_of_floors,footprint_area,living_area,geometry,building_area,population,demand,demand_left,avg_dist,supplyed_demands_within,supplyed_demands_without,min_dist,provison_value
1,True,None,1,NaN,7,609,426.3,"MULTIPOLYGON (((3367578.027 8390970.091, 33676...",609,21.315,0,0.0,NaN,0,0,None,NaN
2,True,None,1,NaN,10,645,451.5,"MULTIPOLYGON (((3370037.390 8390437.108, 33700...",645,22.575,0,0.0,NaN,0,0,0.370117,NaN
3,True,None,1,NaN,10,692,484.4,"MULTIPOLYGON (((3369845.787 8390348.852, 33698...",692,24.220,0,0.0,NaN,0,0,0.819824,NaN
4,True,None,1,NaN,8,3033,2123.1,"MULTIPOLYGON (((3369242.552 8391172.886, 33693...",3033,106.155,0,0.0,NaN,0,0,0.0,NaN
5,True,None,1,NaN,8,3282,2297.4,"MULTIPOLYGON (((3369300.527 8390231.929, 33693...",3282,114.870,0,0.0,NaN,0,0,0.560059,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,True,residential,16,16.0,16,843,9441.6,"MULTIPOLYGON (((3365571.213 8390499.070, 33655...",13488,472.080,3,0.0,0.52,3,0,0.52002,1.0
1027,True,None,1,NaN,12,5354,3747.8,"MULTIPOLYGON (((3367149.209 8390226.899, 33673...",5354,187.390,1,0.0,0.02,1,0,0.020004,1.0
1028,True,None,1,NaN,12,3078,2154.6,"MULTIPOLYGON (((3367404.957 8390139.690, 33674...",3078,107.730,0,0.0,NaN,0,0,0.020004,NaN
1029,True,None,1,NaN,12,1429,1000.3,"MULTIPOLYGON (((3367379.550 8390224.812, 33674...",1429,50.015,0,0.0,NaN,0,0,1.070312,NaN


In [326]:
import pandas as pd

# Проверяем, что атрибут 'provison_value' существует в GeoDataFrame
if 'provison_value' not in build_prov.columns:
    raise ValueError("GeoDataFrame 'build_prov' must contain a 'provison_value' column.")

# Вычисляем среднее значение и медиану
mean_provision = build_prov['provison_value'].mean()
median_provision = build_prov['provison_value'].median()

# Выводим результаты
print(f"Среднее значение обеспечения: {mean_provision:.2f}")
print(f"Медиана обеспечения: {median_provision:.2f}")


Среднее значение обеспечения: 0.99
Медиана обеспечения: 1.00


In [327]:
!pip install folium matplotlib mapclassify


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\meles\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
build_prov.to_file("provision.geojson") #